In [ ]:
def solve_1D_BPP_WB(cargo, deferred_items = None, attempted_combinations = None , number_of_opened_uld = None, packed_ULDs = None, open_new_uld = False, open_new_extra_uld = False, deferred_items_optimal_flag = False):
    """
    Solve 1D Bin Packing Problem combined with Weight and Balance
    """
    m = Model("1D_BPP_WB")

    '''Parameters'''
    
    loadfactor = 0.8
    a_lat_TOW = 0.5
    b_lat_TOW = 0.5
    a_lat_LW = 0.5
    b_lat_LW = 0.5

    '''Open a new extra ULD compared to the actual set of ULDs [Feedback loop]'''

    if open_new_extra_uld is True:
        max_index = max([j.index for j in cargo.uld])
        extra_PMC = ULD(max_index + 1, 'PMC', 'PMC' + '-' + str(max_index + 1))
        extra_AKE = ULD(max_index + 2, 'AKE', 'AKE' + '-' + str(max_index + 2))
        cargo.uld.append(extra_PMC)
        cargo.uld.append(extra_AKE)
        cargo.define_parameters_ULD()


    '''Decision variables'''

    # BUG 4: Variable w innecesaria - Se crea variable continua w[i,j,t] para linearizar peso, pero no es necesaria.
    f = {}
    w = {}  # BUG 4: Esta variable no deberia existir
    u = {}
    p = {}
    z = {}

    
    for j in cargo.uld:
        for t in aircraft.loadlocations:
            f[j.index, t.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'f_{j.index}_{t.index}')

    # BUG 4: Definicion de variable w innecesaria
    for i in cargo.items:
        for j in cargo.uld:
            for t in aircraft.loadlocations:
                w[i.index, j.index, t.index] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f'w_{i.index}_{j.index}_{t.index}')

    for j in cargo.uld:
        u[j.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'u_{j.index}')

    for i in cargo.items:
        for j in cargo.uld:
            p[i.index, j.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'p_{i.index}_{j.index}')

    for i in cargo.items:
        for j in cargo.uld:
            for t in aircraft.loadlocations:
                z[i.index, j.index, t.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'z_{i.index}_{j.index}_{t.index}')
    
    '''Feedback loop actions'''

    for uld, items in packed_ULDs.items():
        for j in cargo.uld:
            if j.isNeitherBAXnorBUPnorT and str(uld) == str(j.serialnumber):
                    u[j.index].lb = 1
                    for i in cargo.items:
                        p[i.index, j.index].lb = 0
                        p[i.index, j.index].ub = 0
                    for item in items:
                        for i in cargo.items:
                            if str(i.serialnumber) == str(item):
                                p[i.index, j.index].lb = 1
                                p[i.index, j.index].ub = 1

    if deferred_items is not None and open_new_uld == False:
        for j, items in deferred_items.items():
            for i in items:
                p[i.index, j.index].lb = 0
                p[i.index, j.index].ub = 0

    m.update()

    '''Model Sense'''

    m.ModelSense = GRB.MINIMIZE

    m.update()

    ''' Objective function 1 --> Maximize the %MAC [Highest Priority] '''

    ZFW_index_obj = m.addVar(vtype=GRB.CONTINUOUS, name="ZFW_index_obj")

    m.addConstr(ZFW_index_obj == aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4)
    
    m.addConstr(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type) <= ZFW_index_obj, "ZFW_index_fwd_constraint")
    m.addConstr(ZFW_index_obj <= aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type), "ZFW_index_aft_constraint")

    MAC_obj = (((aircraft.C * (ZFW_index_obj - aircraft.K)) / aircraft.ZFW) + aircraft.reference_arm - aircraft.lemac) / (aircraft.mac_formula / 100)


    m.setObjectiveN(MAC_obj, index = 0, priority = 6, weight = -1)

    m.update()

    '''Objective function 2 --> Place larger volume item in PMC/PAG ULDs and small volume item in AKE ULDs [Medium Priority]'''
    volume_threshold = data_analysis.threshold_volume_in_AKE() * 1000000

    score_AKE = {}
    score_PMC_PAG = {}

    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            score_AKE[j.index] = m.addVar(vtype=GRB.CONTINUOUS, name=f'score_AKE_{j.index}', lb=0)
            score_PMC_PAG[j.index] = m.addVar(vtype=GRB.CONTINUOUS, name=f'score_PMC_PAG_{j.index}', lb=0)

            m.addConstr(score_AKE[j.index] == quicksum(p[i.index, j.index] * (volume_threshold - i.volume) for i in cargo.items if i.volume < volume_threshold and 'AKE' in j.type))
            m.addConstr(score_PMC_PAG[j.index] == quicksum(p[i.index, j.index] * (volume_threshold - i.volume) for i in cargo.items if i.volume < volume_threshold and ('PMC' in j.type or 'PAG' in j.type)))
    
    obj_volume_preference = quicksum(score_AKE[j.index] + score_PMC_PAG[j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT)
    m.setObjectiveN(obj_volume_preference, index = 1, priority = 5, weight = 1)

    m.update()

    ''' Objective function 3 --> Minimize number of ULDs opened [Medium Priority] '''

    obj2 = quicksum(u[j.index] for j in cargo.uld)
    m.setObjectiveN(obj2, index = 2, priority = 4, weight = 1)

    m.update()

    '''Objective function 4 --> Minimize underutilization of ULDs  [Medium Priority]'''
   
    min_load_factor_threshold = 0.2

    underutilization_penalty = {}
    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            actual_load_factor = quicksum(i.volume * p[i.index, j.index] for i in cargo.items) / j.volume

            loadfactor_underutilization = m.addVar(vtype=GRB.CONTINUOUS, name=f"shortfall_volume_{j.index}", lb=0)
            m.addConstr(loadfactor_underutilization >= min_load_factor_threshold - actual_load_factor)

            underutilization_penalty[j.index] = loadfactor_underutilization
    
    obj_underutilization = quicksum(underutilization_penalty.values())
    m.setObjectiveN(obj_underutilization, index=3, priority=3, weight=1)

    m.update()


    '''Objective function 5 --> Minimize the separation of items with the same serialnumber prefix over different ULDs [Medium Priority]'''

    # BUG 5: Sistema de penalizacion solo cuenta si un ULD es usado o no (binario 0 o 1).
    # No cuenta el NUMERO de ULDs usados por booking. 
    # Ejemplo: booking con 10 piezas split en 5 ULDs tiene penalty=5, igual que split en 2 ULDs (penalty=2).
    # Sistema correcto: usar variables Y[b_i] (integer) y Z[b_i,j] (binary) para contar ULDs por booking.
    # CORRECCION: Ver sistema Y-Z en Model.ipynb lineas 584-617 (DV20-DV21)

    prefix_groups = cargo.get_prefix_groups()

    separation_penalty = {}  # BUG 5: Deberia ser Y (integer) para contar ULDs, no binary penalty
    for prefix, items in prefix_groups.items():
        for j in cargo.uld:
            separation_penalty[prefix, j.index] = m.addVar(vtype=GRB.BINARY, name=f'sep_penalty_{prefix}_{j.index}')

    for prefix, items in prefix_groups.items():
        for j in cargo.uld:
            m.addConstr(quicksum(p[i.index, j.index] for i in items) <= len(items) * separation_penalty[prefix, j.index], name=f'C_separation_{prefix}_{j.index}')

            
    obj3 = quicksum(separation_penalty.values())  # BUG 5: Deberia minimizar sum(Y[b_i])
    m.setObjectiveN(obj3, index = 4, priority = 2, weight = 1)

    m.update()

    
    '''Objective function 6 --> Minimize the proximity score of BAX ULDs [Low Priority]'''
    
    obj4 = quicksum(aircraft.define_proximity_score_loadlocation(t) * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations if j.isBAX)
    m.setObjectiveN(obj4, index = 5, priority = 1, weight = 1)

    m.update()


    '''Constraints'''

    # P1: Apertura de ULDs - Minimum number of ULDs to open (feedback loop)
    m.addConstr(quicksum(u[j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT) >= number_of_opened_uld, name = 'open_new_uld_constraint')

    # P2: Uso de ULD - If ULD is opened, at least one item must be placed in it
    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            m.addConstr(quicksum(p[i.index, j.index] for i in cargo.items) >= u[j.index], name=f'C_new_uld_{j.index}')

    # P3: Capacidad de Peso - Weight of items in ULD cannot exceed ULD max weight capacity
    for j in cargo.uld:
            m.addConstr(quicksum(i.weight * p[i.index, j.index] for i in cargo.items) <= j.max_weight * u[j.index], name = f'C1_{j.index}')

    # P4: Capacidad Volumétrica - Volume of items in ULD cannot exceed ULD volume with loadfactor
    for j in cargo.uld:
            m.addConstr(quicksum(i.volume * p[i.index, j.index] for i in cargo.items) <= j.volume * u[j.index] * loadfactor, name = f'C2_{j.index}')

    # BUG 2: Asignacion incluye BAX/BUP/T - La suma incluye TODOS los ULDs (incluyendo BAX/BUP/T).
    # Los ULDs BAX/BUP/T son contenedores pre-construidos que NO pueden recibir items de optimizacion.
    # Esto puede crear soluciones infactibles o forzar items en contenedores BAX.
    # CORRECCION: Agregar filtro "if j.isNeitherBAXnorBUPnorT" en el quicksum
    # Ver Model.ipynb linea 211: quicksum(p[i.index, j.index] for j in cargo.uld if j.isNeitherBAXnorBUPnorT) == 1

    # P5: Asignación Única - Every item must be placed in exactly one ULD 
    for i in cargo.items:
        m.addConstr(quicksum(p[i.index, j.index] for j in cargo.uld) == 1, name = f'C3_{i.index}')  # BUG 2: Falta filtro isNeitherBAXnorBUPnorT

    # P6: Combinatorial ULD-Position - ULD can only be at a position if it is activated
    for j in cargo.uld:
        m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.loadlocations) == u[j.index], name = f'C_combi_1_{j.index}')

    # P7: Posición Única - Each position can hold at most one ULD
    for t in aircraft.loadlocations:
        m.addConstr(quicksum(f[j.index, t.index] for j in cargo.uld) <= 1, name = f'C4_{t.index}')

    # P8: BAX/BUP/T Asignación - Special ULDs must be assigned to exactly one position
    for j in cargo.uld:
        if j.isBAXorBUPorT:
            m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.loadlocations) == 1, name = f'C_combi_2_{j.index}')

    # BF1: BAX Position Fixed - BAX ULDs fixed to actual positions from real flight data
    for j in cargo.uld:
        if j.isBAX:
            index_position_bax = [t.index for t in aircraft.loadlocations if t.location == j.actual_position_bax][0]
            m.addConstr(f[j.index, index_position_bax]  == 1, name = f'C_BAX_fixed_{j.index}')

    # P9: Prohibición en BAX/BUP/T - No items can be loaded in BAX, BUP, or T ULDs
    for i in cargo.items:
        for j in cargo.uld:
            if j.isBAXorBUPorT:
                m.addConstr(p[i.index, j.index] == 0, name = f'C_combi_3_{i.index}_{j.index}')

    
    # BUG 4: Restricciones de linearizacion L1-L7 innecesarias - Toda esta seccion no deberia existir.
    # Gurobi maneja i.weight * z[i,j,t] nativamente sin necesidad de variable w ni linearizacion.
    # L1: Linearización w <= M*p - w_ijt is 0 if item i is not in ULD j 
    for i in cargo.items:
        for j in cargo.uld:
            for t in aircraft.loadlocations:
                m.addConstr(w[i.index, j.index, t.index] <= M * p[i.index, j.index], name = f'C_lin_1_{i.index}_{j.index}_{t.index}')

    # L2: Linearización w <= M*f - w_ijt is 0 if ULD j is not at position t
    for i in cargo.items:
        for j in cargo.uld:
            for t in aircraft.loadlocations:
                m.addConstr(w[i.index, j.index, t.index] <= M * f[j.index, t.index], name = f'C_lin_2_{i.index}_{j.index}_{t.index}')

    # L3, L4, L5: Linearización z - z_ijt active only if both p_ij and f_jt are active
    for i in cargo.items:
        for j in cargo.uld:
            for t in aircraft.loadlocations:
                m.addConstr(z[i.index, j.index, t.index] <= p[i.index, j.index], name = f'C_lin_3_{i.index}_{j.index}_{t.index}')
                m.addConstr(z[i.index, j.index, t.index] <= f[j.index, t.index], name = f'C_lin_4_{i.index}_{j.index}_{t.index}')
                m.addConstr(z[i.index, j.index, t.index] >= p[i.index, j.index] + f[j.index, t.index] - 1, name = f'C_lin_5_{i.index}_{j.index}_{t.index}')

    # L6: Linearización w >= wi - M(1-z) - w_ijt equals actual weight when z_ijt is 1
    for i in cargo.items:
        for j in cargo.uld:
            for t in aircraft.loadlocations:
                m.addConstr(w[i.index, j.index, t.index] >= i.weight - M * (1 - z[i.index, j.index, t.index]), name = f'C_lin_6_{i.index}_{j.index}_{t.index}')


    # L7: Linearización w <= wi - Upper bound on w_ijt 
    for i in cargo.items:
        for j in cargo.uld:
            for t in aircraft.loadlocations:
                m.addConstr(w[i.index, j.index, t.index] <= i.weight, name = f'C_lin_7_{i.index}_{j.index}_{t.index}')

    # P10: Posiciones Prohibidas - ULD type must match position type compatibility
    for j in cargo.uld:
        m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.define_forbidden_positions_for_ULD(j)) == 0, name = f'C5_{j.index}')

    #Because one type of ULDs’ predefined positions may overlap with others, one of these overlapping positions is occupied, and the others can no longer be allocated. 
    for j_1 in cargo.uld:
        for j_2 in cargo.uld:
            if j_1 != j_2:
                for t_1 in aircraft.loadlocations:
                        for t_2 in aircraft.define_overlapping_positions(t_1):
                            m.addConstr(f[j_1.index, t_1.index] + f[j_2.index, t_2.index] <= 1, name = f'C6_{j_1.index}_{j_2.index}_{t_1.index}_{t_2.index}')

    # BUG 8: Es redundante la estriccion de peso por posicion dividida
    # C7_1: peso de items <= max_weight_position
    # C7_2: peso de ULDs BAX <= max_weight_position
    # CORRECCION: Combinar en UNA sola restriccion: peso_items + peso_BAX <= max_weight_position

    # P12: Peso por Posición - Weight at position cannot exceed position limit
    for t in aircraft.loadlocations:
        m.addConstr(quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld)
                    <= aircraft.define_max_weight_postion(t), name = f'C7_1_{t.index}')  # BUG 8: Deberia combinarse con C7_2
        
    for t in aircraft.loadlocations:
        m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT)
                    <= aircraft.define_max_weight_postion(t), name = f'C7_2_{t.index}')  # BUG 8: Deberia combinarse con C7_1

    # BUG 1: debe sumar en compartimiento, esto tb esta en todos los modelos (P13,..., P16), no es para cada t, sino para cada compartimiento, y sumar las t.
    # P13-P16: Peso por Compartimento - Total weight per compartment limits. Ya solucionado
    m.addConstr(
        quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT for t in aircraft.loadlocations_C1)
        <= aircraft.max_weight_C1,
        name='C_Added_1'
    )

    m.addConstr(
        quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT for t in aircraft.loadlocations_C2)
        <= aircraft.max_weight_C2,
        name='C_Added_2'
    )

    m.addConstr(
        quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT for t in aircraft.loadlocations_C3)
        <= aircraft.max_weight_C3,
        name='C_Added_3'
    )

    m.addConstr(
        quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT for t in aircraft.loadlocations_C4)
        <= aircraft.max_weight_C4,
        name='C_Added_4'
    )
        
    # P17: Peso Compartimentos C1+C2 - Combined weight in front compartments
    m.addConstr(
        quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1_C2) +
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT for t in aircraft.loadlocations_C1_C2)
        <= aircraft.max_weight_C1_C2,
        name='C_Added_5'
    )
        
    # P18: Peso Compartimentos C3+C4 - Combined weight in rear compartments
    m.addConstr(
        quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3_C4) +
        quicksum(j.weight * f[j.index, t.index] for j in cargo.uld if j.isBAXorBUPorT for t in aircraft.loadlocations_C3_C4)
        <= aircraft.max_weight_C3_C4,
        name='C_Added_6'
    )
        
    # P19: Peso Total (MPL) - Total cargo weight cannot exceed Maximum Payload Limit
    m.addConstr(quicksum(w[i.index, j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld for i in cargo.items) +
                quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT)
                <= aircraft.define_MPL(), name = f'C8')

    # P20-P21 & P22-P23: Balance Lateral - Lateral balance constraints for TOW and LW
    m.addConstr((quicksum(w[i.index, j_1.index, t_left.index] for i in cargo.items for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) +
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld if j_1.isBAXorBUPorT for t_left in aircraft.loadlocations_left)) - 
                (quicksum(w[i.index, j_2.index, t_right.index] for i in cargo.items for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) +
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld if j_2.isBAXorBUPorT for t_right in aircraft.loadlocations_right)) <= 
                a_lat_TOW * (quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations) + 
                            quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT) + aircraft.OEW + aircraft.TOF) * b_lat_TOW, name = f'C9_1')
    
    m.addConstr((quicksum(w[i.index, j_2.index, t_right.index] for i in cargo.items for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) +
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld if j_2.isBAXorBUPorT for t_right in aircraft.loadlocations_right)) - 
                (quicksum(w[i.index, j_1.index, t_left.index] for i in cargo.items for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) +
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld if j_1.isBAXorBUPorT for t_left in aircraft.loadlocations_left)) <= 
                a_lat_TOW * (quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations) +
                            quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT) + aircraft.OEW + aircraft.TOF) * b_lat_TOW, name = f'C9_2')

    # P20-P21 & P22-P23: Balance Lateral - Lateral balance constraints for TOW and LW
    m.addConstr((quicksum(w[i.index, j_1.index, t_left.index] for i in cargo.items for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) +
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld if j_1.isBAXorBUPorT for t_left in aircraft.loadlocations_left)) - 
                (quicksum(w[i.index, j_2.index, t_right.index] for i in cargo.items for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) +
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld if j_2.isBAXorBUPorT for t_right in aircraft.loadlocations_right)) <= 
                a_lat_LW * (quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations) +
                            quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT) + aircraft.OEW + aircraft.TOF - aircraft.TripF) * b_lat_LW, name = f'C10_1')

    m.addConstr((quicksum(w[i.index, j_2.index, t_right.index] for i in cargo.items for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) +
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld if j_2.isBAXorBUPorT for t_right in aircraft.loadlocations_right)) - 
                (quicksum(w[i.index, j_1.index, t_left.index] for i in cargo.items for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) +
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld if j_1.isBAXorBUPorT for t_left in aircraft.loadlocations_left)) <= 
                a_lat_LW * (quicksum(w[i.index, j.index, t.index] for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations) +
                            quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld if j.isBAXorBUPorT) + aircraft.OEW + aircraft.TOF - aircraft.TripF) * b_lat_LW, name = f'C10_2')

    # P24-P25: Envelope CG TOW - Longitudinal CG envelope for takeoff (forward and aft limits)
    m.addConstr(aircraft.define_INDEX_TOW_fwd(aircraft.aircraft_type) <= aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX() +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4, name = 'C11_1')
    
    m.addConstr(aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX() +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4 <= 
                aircraft.define_INDEX_TOW_aft(aircraft.aircraft_type), name = 'C11_2')
    
    # P26-P27: Envelope CG ZFW - Longitudinal CG envelope for zero fuel weight (forward and aft limits)
    m.addConstr(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type) <= aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4, name = 'C12_1')
    
    m.addConstr(aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C1 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C2 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C3 +
                (quicksum(w[i.index, j.index, t.index]  for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) +
                quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT)) * aircraft.delta_index_cargo_C4 <= 
                aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type), name = 'C12_2')

    # P28: Manejo Especial COL/CRT - COL and CRT items cannot be in same ULD
    for j in cargo.uld:
        if j.isNeitherBAXnorBUPnorT:
            for i_1 in cargo.items:
                for i_2 in cargo.items:
                    if i_1 != i_2:
                        if i_1.COL == 1 and i_2.CRT == 1:
                            m.addConstr(p[i_1.index, j.index] + p[i_2.index, j.index] <= 1, name = f'C_special_1_{i_1.index}_{i_2.index}_{j.index}')
                            

    # P29-P30: COL/CRT Compartimentos - COL and CRT cargo cannot be in same compartment (aircraft-specific)
    COL_items_present = any(i.COL == 1 for i in cargo.items)
    CRT_items_present = any(i.CRT == 1 for i in cargo.items)
    COL_items_indices = [i.index for i in cargo.items if i.COL == 1]
    CRT_items_indices = [i.index for i in cargo.items if i.CRT == 1]
    T_with_COL = [j.index for j in cargo.uld if j.COL == 1]
    T_with_CRT = [j.index for j in cargo.uld if j.CRT == 1]

    # CORRECCIÓN COL/CRT: Control por compartimento con lógica Big-M

    col_items = [i for i in cargo.items if i.COL == 1]
    crt_items = [i for i in cargo.items if i.CRT == 1]
    uld_with_COL_flag = [j for j in cargo.uld if getattr(j, 'COL', 0) == 1]
    uld_with_CRT_flag = [j for j in cargo.uld if getattr(j, 'CRT', 0) == 1]

    if str(aircraft.aircraft_type) in ['772', '77W']:
        compartments = {
            'front': aircraft.loadlocations_C1_C2,
            'aft': aircraft.loadlocations_C3_C4,
        }

        for label, load_locations in compartments.items():
            COL_k = m.addVar(vtype=GRB.BINARY, name=f'COL_{label}')
            CRT_k = m.addVar(vtype=GRB.BINARY, name=f'CRT_{label}')

            if col_items:
                m.addConstr(
                    quicksum(z[i.index, j.index, t.index] for i in col_items for j in cargo.uld if j.isNeitherBAXnorBUPnorT for t in load_locations)
                    <= len(col_items) * COL_k,
                    name=f'C_COL_items_{label}'
                )
            if uld_with_COL_flag:
                m.addConstr(
                    quicksum(f[j.index, t.index] for j in uld_with_COL_flag for t in load_locations)
                    <= len(uld_with_COL_flag) * COL_k,
                    name=f'C_COL_uld_{label}'
                )
            if not col_items and not uld_with_COL_flag:
                m.addConstr(COL_k == 0, name=f'C_COL_presence_{label}_zero')

            if crt_items:
                m.addConstr(
                    quicksum(z[i.index, j.index, t.index] for i in crt_items for j in cargo.uld if j.isNeitherBAXnorBUPnorT for t in load_locations)
                    <= len(crt_items) * CRT_k,
                    name=f'C_CRT_items_{label}'
                )
            if uld_with_CRT_flag:
                m.addConstr(
                    quicksum(f[j.index, t.index] for j in uld_with_CRT_flag for t in load_locations)
                    <= len(uld_with_CRT_flag) * CRT_k,
                    name=f'C_CRT_uld_{label}'
                )
            if not crt_items and not uld_with_CRT_flag:
                m.addConstr(CRT_k == 0, name=f'C_CRT_presence_{label}_zero')

            m.addConstr(COL_k + CRT_k <= 1, name=f'C_COL_CRT_conflict_{label}')

    if str(aircraft.aircraft_type) in ['789', '781']:
        if col_items:
            m.addConstr(
                quicksum(z[i.index, j.index, t.index] for i in col_items for j in cargo.uld if j.isNeitherBAXnorBUPnorT for t in aircraft.loadlocations_C3_C4) == 0,
                name='C_COL_items_aft_787'
            )
        if uld_with_COL_flag:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in uld_with_COL_flag for t in aircraft.loadlocations_C3_C4) == 0,
                name='C_COL_uld_aft_787'
            )

        if crt_items:
            m.addConstr(
                quicksum(z[i.index, j.index, t.index] for i in crt_items for j in cargo.uld if j.isNeitherBAXnorBUPnorT for t in aircraft.loadlocations_C3_C4) == 0,
                name='C_CRT_items_aft_787'
            )
        if uld_with_CRT_flag:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in uld_with_CRT_flag for t in aircraft.loadlocations_C3_C4) == 0,
                name='C_CRT_uld_aft_787'
            )

        COL_front = m.addVar(vtype=GRB.BINARY, name='COL_front_787')
        CRT_front = m.addVar(vtype=GRB.BINARY, name='CRT_front_787')

        if col_items:
            m.addConstr(
                quicksum(z[i.index, j.index, t.index] for i in col_items for j in cargo.uld if j.isNeitherBAXnorBUPnorT for t in aircraft.loadlocations_C1_C2)
                <= len(col_items) * COL_front,
                name='C_COL_items_front_787'
            )
        if uld_with_COL_flag:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in uld_with_COL_flag for t in aircraft.loadlocations_C1_C2)
                <= len(uld_with_COL_flag) * COL_front,
                name='C_COL_uld_front_787'
            )
        if not col_items and not uld_with_COL_flag:
            m.addConstr(COL_front == 0, name='C_COL_front_787_zero')

        if crt_items:
            m.addConstr(
                quicksum(z[i.index, j.index, t.index] for i in crt_items for j in cargo.uld if j.isNeitherBAXnorBUPnorT for t in aircraft.loadlocations_C1_C2)
                <= len(crt_items) * CRT_front,
                name='C_CRT_items_front_787'
            )
        if uld_with_CRT_flag:
            m.addConstr(
                quicksum(f[j.index, t.index] for j in uld_with_CRT_flag for t in aircraft.loadlocations_C1_C2)
                <= len(uld_with_CRT_flag) * CRT_front,
                name='C_CRT_uld_front_787'
            )
        if not crt_items and not uld_with_CRT_flag:
            m.addConstr(CRT_front == 0, name='C_CRT_front_787_zero')

        m.addConstr(COL_front + CRT_front <= 1, name='C_COL_CRT_conflict_front_787')


    '''Optimize the model'''

    if not deferred_items_optimal_flag:
        WB_env = m.getMultiobjEnv(0)
        volume_env = m.getMultiobjEnv(1)
        uld_env = m.getMultiobjEnv(2)
        underutilization_env = m.getMultiobjEnv(3)
        separation_env = m.getMultiobjEnv(4)
        bax_env = m.getMultiobjEnv(5)

        WB_env.setParam(GRB.Param.TimeLimit, 60)
        volume_env.setParam(GRB.Param.TimeLimit, 15)
        uld_env.setParam(GRB.Param.TimeLimit, 15)
        separation_env.setParam(GRB.Param.TimeLimit, 15)
        underutilization_env.setParam(GRB.Param.TimeLimit, 15)
        bax_env.setParam(GRB.Param.TimeLimit, 15)

        m.optimize()
    
    else:
        m.setParam(GRB.Param.MIPGap, 0.1)
        WB_env = m.getMultiobjEnv(0)
        volume_env = m.getMultiobjEnv(1)
        uld_env = m.getMultiobjEnv(2)
        underutilization_env = m.getMultiobjEnv(3)
        separation_env = m.getMultiobjEnv(4)
        bax_env = m.getMultiobjEnv(5)

        WB_env.setParam(GRB.Param.TimeLimit, 60)
        volume_env.setParam(GRB.Param.TimeLimit, 15)
        uld_env.setParam(GRB.Param.TimeLimit, 15)
        separation_env.setParam(GRB.Param.TimeLimit, 15)
        underutilization_env.setParam(GRB.Param.TimeLimit, 15)
        bax_env.setParam(GRB.Param.TimeLimit, 15)

        m.optimize()

    status = m.status
    if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE: 
        print('The model is infeasible or unbounded')
        # m.computeIIS()
        # m.write('model.ilp')
        print('Infeasibility report written to model.ilp')
    elif status == GRB.Status.TIME_LIMIT:
        print('Time limit reached')
    elif status == GRB.Status.OPTIMAL:
        print('Solution found')
    elif status == GRB.Status.INTERRUPTED:
        print('Optimization was stopped early')

    '''Results'''
    results_filename = 'Results.txt'
    folder_path = project_setup.setup_project_directory(aircraft.flight_number, aircraft.date, aircraft.departure_airport, aircraft.arrival_airport, baseline = False, optimized_actual = False, BAX_fixed = True)
    results_file_path = os.path.join(folder_path, results_filename)
    loadfactor_dict = {}

    with open(results_file_path, 'w') as file:
        total_weight = 0
        number_of_uld_solution = 0
        for t in aircraft.loadlocations:
            for j in cargo.uld:
                if f[j.index, t.index].x > 0.9999:
                    if j.isNeitherBAXnorBUPnorT:
                        number_of_uld_solution += 1
                        print(f'ULD {j.serialnumber} with weight {sum(i.weight * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999):.1f} kg with a volume loadfactor of {(sum(i.volume * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.volume) * 100:.3f}% and a weight loadfactor of {(sum(i.weight * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.max_weight) * 100:.3f}% is loaded to position {t.location}')
                        file.write(f'ULD {j.serialnumber} with weight {sum(i.weight * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999):.1f} kg with a volume loadfactor of {(sum(i.volume * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.volume) * 100:.3f}% and a weight loadfactor of {(sum(i.weight * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.max_weight) * 100:.3f}% is loaded to position {t.location}\n')
                        volume_loadfactor = (sum(i.volume * p[i.index, j.index].x for i in cargo.items if p[i.index, j.index].x > 0.9999) / j.volume)
                        loadfactor_dict[j] = volume_loadfactor
                        total_weight += sum(i.weight * p[i.index, j.index].x for i in cargo.items)
                    if j.isBAXorBUPorT:
                        print(f'ULD {j.serialnumber} with weight {j.weight} kg is loaded to position {t.location}')
                        file.write(f'ULD {j.serialnumber} with weight {j.weight} kg is loaded to position {t.location}\n')
                        total_weight += j.weight

        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'Weight in Compartment 1: {sum(w[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT):.1f} kg')
        file.write(f'Weight in Compartment 1: {sum(w[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C1) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C1 if j.isBAXorBUPorT):.1f} kg\n')
        print(f'Weight in Compartment 2: {sum(w[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT):.1f} kg')
        file.write(f'Weight in Compartment 2: {sum(w[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C2) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C2 if j.isBAXorBUPorT):.1f} kg\n')
        print(f'Weight in Compartment 3: {sum(w[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT):.1f} kg')
        file.write(f'Weight in Compartment 3: {sum(w[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C3) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C3 if j.isBAXorBUPorT):.1f} kg\n')
        print(f'Weight in Compartment 4: {sum(w[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT):.1f} kg')
        file.write(f'Weight in Compartment 4: {sum(w[i.index, j.index, t.index].x for i in cargo.items for j in cargo.uld for t in aircraft.loadlocations_C4) + sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C4 if j.isBAXorBUPorT):.1f} kg\n')

        actual_builded = [j for j in cargo.uld if not j.isBAXorBUPorT]

        #Calculating %MAC
        TOW_index = aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX()
        ZFW_index = aircraft.DOI + aircraft.define_INDEX_PAX()

        dict_loadlocations = {aircraft.delta_index_cargo_C1: aircraft.loadlocations_C1, aircraft.delta_index_cargo_C2: aircraft.loadlocations_C2, aircraft.delta_index_cargo_C3: aircraft.loadlocations_C3, aircraft.delta_index_cargo_C4: aircraft.loadlocations_C4}

        for i in cargo.items:
            for j in cargo.uld:
                for value, compartment in dict_loadlocations.items():
                    for t in compartment:
                        if f[j.index, t.index].x > 0.9999 and j.isNeitherBAXnorBUPnorT:
                            TOW_index += w[i.index, j.index, t.index].x  * float(value)
                            ZFW_index += w[i.index, j.index, t.index].x  * float(value)

        for j in cargo.uld:
            for value, compartment in dict_loadlocations.items():
                for t in compartment:
                    if f[j.index, t.index].x > 0.9999 and j.isBAXorBUPorT:
                        TOW_index += j.weight * f[j.index, t.index].x * float(value) 
                        ZFW_index += j.weight * f[j.index, t.index].x * float(value)

        increment_value = aircraft.define_ff_increment_MAC_ZFW(MAC_obj.getValue())
        fuel_saving_kg = aircraft.TripF * (increment_value / 100)

        print('===========================================================================')
        file.write('===========================================================================\n')
        print(f'%MAC ZFW is {MAC_obj.getValue()}')
        file.write(f'%MAC ZFW is {MAC_obj.getValue()}\n')
        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'The actual %MAC ZFW for this flight was {aircraft.actual_MAC_ZFW}')
        file.write(f'The actual %MAC ZFW for this flight was {aircraft.actual_MAC_ZFW}\n')
        print(f'Resulting in a fuel deviation of {increment_value:.3f}% or {fuel_saving_kg:.3f} kg')
        file.write(f'Resulting in a fuel deviation of {increment_value:.3f}% or {fuel_saving_kg:.3f} kg\n')
        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'{number_of_uld_solution} ULDs are built by the model')
        file.write(f'{number_of_uld_solution} ULDs are built by the model\n')
        print(f'{cargo.total_number_of_build_ULDs} ULDs were actually built')
        file.write(f'{cargo.total_number_of_build_ULDs} ULDs were actually built\n')
        print('---------------------------------------------------------------------------')
        print(f'The ULDs are builded and blocked in the model:')
        for uld, items in packed_ULDs.items():
            print(f'{str(uld)}')

    #Making results dictionaries
    results_1D_BPP_WB = {}

    for j in cargo.uld:
        if u[j.index].x == 1:
            if j.isNeitherBAXnorBUPnorT:
                results_1D_BPP_WB[j] = []

    for i in cargo.items:
        for j in cargo.uld:
            if p[i.index, j.index].x > 0.999:
                results_1D_BPP_WB[j].append(i)

    number_of_items_in_results = 0
    for j, items in results_1D_BPP_WB.items():
        number_of_items_in_results += len(items)
    print('---------------------------------------------------------------------------')
    
    return results_1D_BPP_WB, ZFW_index, TOW_index, loadfactor_dict

In [48]:
def solve_3D_BPP(results_1D_BPP_WB):
    deferred_items = {}
    all_placed_items = {}
    all_extreme_points = {}

    for j, items in results_1D_BPP_WB.items():
        extreme_points = EP.get_starting_extreme_points(j)
        items_to_place = items.copy()
        placed_items = {}
        deferred_items[j] = []

        while items_to_place:
            next_item, placement_details, defer_reason = EP.find_best_next_item_and_placement(items_to_place, j, placed_items, extreme_points)

            if next_item:
                best_ep, best_orientation, best_merit, best_support_count = placement_details
                placed_items, added_points, removed_points = EP.place_item(next_item, best_ep, placed_items, best_orientation, extreme_points)
                extreme_points = EP.update_extreme_points(extreme_points, placed_items[next_item.serialnumber], next_item, j)
                items_to_place.remove(next_item)

            else:
                print(f'Items deferred for ULD {j.serialnumber}:')
                for item in items_to_place:
                    print(f'{item.serialnumber} deferred due to {defer_reason}')
                    deferred_items[j].append(item)
                print('---------------------------------------------------------------------------')

                items_to_place = []

        all_placed_items[j] = placed_items
        all_extreme_points[j] = extreme_points

    return deferred_items, all_placed_items, all_extreme_points


In [ ]:
def feedback_loop(cargo):
    start_time = time.time()
    total_time_1D_BPP_WB = 0
    total_time_3D_BPP = 0
    iteration = 1
    number_of_opened_uld = 0
    open_new_uld = False 
    open_new_extra_uld = False
    extra_uld_opened_in_previous_iteration = False
    solution_found = False
    deferred_items = {}
    previous_deferred_items = {}
    packed_ULDs = {}
    attempted_combinations = set()

    number_of_uld_actually_used = len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT])
    color_map = EP.get_color_map(cargo.items)
    folder_path = project_setup.setup_project_directory(aircraft.flight_number, aircraft.date, aircraft.departure_airport, aircraft.arrival_airport, baseline = False, optimized_actual = False, BAX_fixed = True)


    while not solution_found:
        print(f'Iteration {iteration}')
        print('===========================================================================')
        print(f'Starting optimization for iteration {iteration}')
        start_time_1D_BPP_WB = time.time()
        results_1D_BPP_WB_optimal, ZFW_index_optimal, TOW_index_optimal, loadfactor_dict_optimal = solve_1D_BPP_WB(cargo, deferred_items = deferred_items, attempted_combinations = attempted_combinations, 
                                                                                                                   number_of_opened_uld = number_of_opened_uld, packed_ULDs = packed_ULDs, 
                                                                                                                   open_new_uld = open_new_uld, open_new_extra_uld = open_new_extra_uld, 
                                                                                                                   deferred_items_optimal_flag = False)
        open_new_extra_uld = False
        total_time_1D_BPP_WB += time.time() - start_time_1D_BPP_WB
        number_of_opened_uld = sum(1 for value in results_1D_BPP_WB_optimal.values() if value)

        start_time_3D_BPP = time.time()
        deferred_items_optimal, placed_items_optimal, extreme_points_optimal = solve_3D_BPP(results_1D_BPP_WB_optimal)
        total_time_3D_BPP += time.time() - start_time_3D_BPP

        for j, items in deferred_items_optimal.items():
            if ((len(items) == 0) and (loadfactor_dict_optimal[j] >= 0.65)) or ((loadfactor_dict_optimal[j] >= 0.75) and (len(deferred_items_optimal[j]) <= 2)):
                if len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT]) - len(packed_ULDs) > 2:
                    packed_ULDs[j.serialnumber] = list(placed_items_optimal[j].keys())

        number_of_deferred_items_optimal = sum(len(items) for items in deferred_items_optimal.values())

        if number_of_deferred_items_optimal == 0:
            solution_found = True
            print('Optimal solution found with no deferred items')
            print('===========================================================================')
            plot.WB(aircraft, ZFW_index_optimal, TOW_index_optimal, folder_path)
            plot.BPP(cargo, results_1D_BPP_WB_optimal, placed_items_optimal, extreme_points_optimal, color_map, folder_path)
            end_time = time.time()
            break

        deferred_items = deferred_items_optimal
        current_iteration_deferred_flag = False
        
        for j, items in deferred_items_optimal.items():
            for i in items:
                previous_deferred_items[i.serialnumber] = previous_deferred_items.get(i.serialnumber, 0) + 1

                if ((previous_deferred_items[i.serialnumber] >= (number_of_opened_uld - len(packed_ULDs.keys()))) or 
                    ((number_of_opened_uld < len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT])) and (sum(1 for value in deferred_items_optimal.values() if value) == 1)) and
                    not extra_uld_opened_in_previous_iteration):

                    print(f'Flag set: opening a new ULD')
                    print(f'Item {i.serialnumber} is causing the flag')
                    print('---------------------------------------------------------------------------')
                    current_iteration_deferred_flag = True
                    if number_of_opened_uld < number_of_uld_actually_used:
                        number_of_opened_uld = number_of_opened_uld + 1
                        previous_deferred_items = {k: 0 for k in previous_deferred_items.keys()} 
                        extra_uld_opened_in_previous_iteration = False
                    elif number_of_opened_uld >= number_of_uld_actually_used:
                        open_new_extra_uld = True
                        number_of_opened_uld = number_of_opened_uld + 1
                        print(number_of_opened_uld)
                        previous_deferred_items = {k: 0 for k in previous_deferred_items.keys()}
                        extra_uld_opened_in_previous_iteration = True
                    break

        print(f'{number_of_deferred_items_optimal} items have been deferred this iteration')
        print(f'{sum(1 for value in deferred_items_optimal.values() if value)} ULDs have deferred items this iteration')

        if not current_iteration_deferred_flag and (sum(1 for value in deferred_items_optimal.values() if value) > 1):
            print(f'Continuing with optimal strategy due to high number of deferred items')
            print('===========================================================================')
            open_new_uld = False

        if not current_iteration_deferred_flag and (sum(1 for value in deferred_items_optimal.values() if value) == 1):
            print(f'Continuing with suboptimal startegy')
            print('===========================================================================')
            start_time_1D_BPP_WB_suboptimal = time.time()
            results_1D_BPP_WB_suboptimal, ZFW_index_suboptimal, TOW_index_suboptimal, loadfactor_dict_suboptimal = solve_1D_BPP_WB(cargo, deferred_items = deferred_items, attempted_combinations = attempted_combinations, 
                                                                                                                                   number_of_opened_uld = number_of_opened_uld, packed_ULDs = packed_ULDs, open_new_uld = open_new_uld, 
                                                                                                                                   open_new_extra_uld = open_new_extra_uld, deferred_items_optimal_flag = True)
            total_time_1D_BPP_WB += time.time() - start_time_1D_BPP_WB_suboptimal


            start_time_3D_BPP_suboptimal = time.time()
            deferred_items_suboptimal, placed_items_suboptimal, extreme_points_suboptimal = solve_3D_BPP(results_1D_BPP_WB_suboptimal)
            total_time_3D_BPP += time.time() - start_time_3D_BPP_suboptimal

            for j, items in deferred_items_suboptimal.items():
                if ((len(items) == 0) and (loadfactor_dict_suboptimal[j] >= 0.65)) or ((loadfactor_dict_suboptimal[j] >= 0.75) and (len(deferred_items_suboptimal[j]) <= 2)):
                    if len([j for j in cargo.uld if j.isNeitherBAXnorBUPnorT]) - len(packed_ULDs) > 2:
                        packed_ULDs[j.serialnumber] = list(placed_items_suboptimal[j].keys())

            number_of_deferred_items_suboptimal = sum(len(items) for items in deferred_items_suboptimal.values())
            
            if number_of_deferred_items_suboptimal == 0:
                solution_found = True
                print('Suboptimal solution found with no deferred items')
                print('===========================================================================')
                plot.WB(aircraft, ZFW_index_suboptimal, TOW_index_suboptimal, folder_path)
                plot.BPP(cargo, results_1D_BPP_WB_suboptimal, placed_items_suboptimal, extreme_points_suboptimal, color_map, folder_path)
                end_time = time.time()
                break

            for j, items in deferred_items_suboptimal.items():
                combination = (j, tuple(results_1D_BPP_WB_suboptimal[j]))
                attempted_combinations.add(combination)

            deferred_items = deferred_items_suboptimal

            print(f'{number_of_deferred_items_suboptimal} items have been deferred this suboptimal optimization')
            print('===========================================================================')

        iteration += 1

    model_info_path = os.path.join(folder_path, 'Model_Information.txt')

    with open(model_info_path, 'w') as file:
        file.write(f'Number of Iteration: {iteration}\n')
        file.write('---------------------------------------------------------------------------\n')
        file.write(f'Total time: {end_time - start_time:.3f} seconds\n')
        file.write('---------------------------------------------------------------------------\n')
        file.write(f'Total time 1D BPP WB: {total_time_1D_BPP_WB:.3f} seconds\n')
        file.write('---------------------------------------------------------------------------\n')
        file.write(f'Total time 3D BPP: {total_time_3D_BPP:.3f} seconds\n')

    return 

In [ ]:
feedback_loop(cargo)